In [1]:
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login
import os
import torch

In [2]:
model_name = "../Llama-3.1-8B"

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

: 

In [ ]:
import torch

# Prepare the prompt
prompt = "What are the future implications of AI?"
inputs = tokenizer(prompt, return_tensors="pt")

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
inputs = inputs.to(device)

# Generate text
outputs = model.generate(
    **inputs,
    max_length=200,   # Adjust the maximum length of the output
    temperature=0.7,  # Adjust for randomness
    top_p=0.95,       # Nucleus sampling
    do_sample=True
)

# Decode and print the generated text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)
